In [6]:
import mysql.connector as mariadb
import pandas as pd

In [7]:
dbConnection = mariadb.connect(host="localhost",user="root",password="password",database="creditcard_capstone")
print("Connected to MariaDb Database...")

Connected to MariaDb Database...


In [8]:
cursor = dbConnection.cursor()

2.1 Transaction Details Module

In [74]:
# 1. Display the transactions made by customers living in a given zip code for a given month 
# and year. Order by day in descending order.

def getTransactionsForCustomersByZipCodeAndDate(zipcode, month, year):

    sqlStatement = "SELECT tran.* FROM cdw_sapp_credit_card tran \
        INNER JOIN cdw_sapp_customer cust \
            ON cust.SSN = tran.CUST_SSN \
        WHERE cust.CUST_ZIP = '{}' \
            AND MONTH(STR_TO_DATE(tran.TIMEID, '%Y%m%d')) = {} \
            AND YEAR(STR_TO_DATE(tran.TIMEID, '%Y%m%d')) = {} \
        ORDER BY DAY(STR_TO_DATE(tran.TIMEID, '%Y%m%d')) DESC"

    cursor.execute(sqlStatement.format(zipcode, month, year))

    result = cursor.fetchall()
    print(result)

In [ ]:
zipcode = input("Enter a zipcode: ")
year = input("Enter the year for which you would like the transaction records: ")
month = input("Enter the month for which you would like the transaction records in a numberic format (1-12):")
getTransactionsForCustomersByZipCodeAndDate(zipcode, month, year)

In [29]:
# 2. Display the number and total values of transactions for a given type. 

def getNumberAndTotalValueOfTransactionsByType(transaction_type):

    sqlStatement1 = "SELECT TRANSACTION_TYPE, COUNT(TRANSACTION_TYPE) AS NUMBER_OF_TRANSACTIONS, \
        round(SUM(TRANSACTION_VALUE),2) AS TOTAL_VALUE_OF_TRANSACTIONS \
            FROM cdw_sapp_credit_card tran \
        WHERE TRANSACTION_TYPE = '{}'"

    cursor.execute(sqlStatement1.format(transaction_type))

    result1 = cursor.fetchall()
    print(result1)

In [30]:
transaction_type = input("Enter a transaction_type: ")
getNumberAndTotalValueOfTransactionsByType(transaction_type)

[('Grocery', 13098, 674103.26)]


In [79]:
# 3. Display the total number and total values of transactions for branches in a given state.

def getTotalNumberAndTotalValueOfTransactionsByState(state):

    sqlStatement2 = "SELECT tran.TRANSACTION_TYPE, \
        COUNT(tran.TRANSACTION_TYPE) AS NUMBER_OF_TRANSACTIONS, \
        round(SUM(tran.TRANSACTION_VALUE),1) AS TOTAL_VALUE_OF_TRANSACTIONS \
            FROM cdw_sapp_credit_card tran \
        INNER JOIN cdw_sapp_branch branch \
            ON tran.BRANCH_CODE = branch.BRANCH_CODE \
        WHERE branch.BRANCH_STATE = '{}' \
        GROUP BY tran.TRANSACTION_TYPE"
    
    cursor.execute(sqlStatement2.format(state))

    result2 = cursor.fetchall()
    print(result2)

In [80]:
state = input("Enter a State(for example CA, TX, FL, etc): ")
getTotalNumberAndTotalValueOfTransactionsByState(state)

[('Bills', 2288, 121599.4), ('Education', 2332, 119299.8), ('Entertainment', 2376, 120061.8), ('Gas', 2192, 111937.8), ('Grocery', 2528, 127127.5), ('Healthcare', 2292, 116796.8), ('Test', 2416, 123661.0)]


2.2 Customer Details Module

In [3]:
# 1. Check the existing account details of a customer.

def getAccountDetailsOfCustomer(first_name, last_name, last_4_digits_SSN):
    sqlStatement3 = "SELECT FIRST_NAME, LAST_NAME, RIGHT(SSN,4) AS LAST_4_digits_of_SSN \
        FROM cdw_sapp_customer \
        WHERE FIRST_NAME = '{}' AND LAST_NAME = '{}' AND RIGHT(SSN,4) = '{}' \
        GROUP BY CONCAT(FIRST_NAME, ' ', LAST_NAME)"
    
    cursor.execute(sqlStatement3.format(first_name, last_name, last_4_digits_SSN))

    result3 = cursor.fetchall()
    print(result3)

In [9]:
first_name = input("Enter First Name: ")
last_name = input("Enter Last Name: ")
last_4_digits_SSN = input("Enter last 4 digits of SSN: ")
getAccountDetailsOfCustomer(first_name, last_name, last_4_digits_SSN)

[('Janelle', 'Stone', '3191')]


In [22]:
# 2. Modify the existing account details of a customer.

def getCustomerAccountDetailsChanged(first_name, last_4_digits_SSN, middle_name):
    sqlStatement4 = "UPDATE cdw_sapp_customer \
        SET MIDDLE_NAME = '{}' \
        WHERE FIRST_NAME = '{}' AND RIGHT(SSN, 4) = '{}'"

    resultSql = "SELECT * FROM cdw_sapp_customer \
        WHERE FIRST_NAME = '{}' AND RIGHT(SSN, 4) = '{}'"
    
    cursor.execute(sqlStatement4.format(middle_name, first_name, last_4_digits_SSN))
    cursor.execute(resultSql.format(first_name, last_4_digits_SSN))

    result4 = cursor.fetchall()
    print(result4)

In [23]:
first_name = input("Enter First Name: ")
last_4_digits_SSN = input("Enter last 4 digits of SSN: ")
middle_name = input("Enter New Middle Name: ")
getCustomerAccountDetailsChanged(first_name, last_4_digits_SSN, middle_name)

[('4210653316211933', 'Andover', 'United States', 'JStone@example.com', '(682)123-7809', 'MA', '01810', 'Janelle', 'Stone', '2018-04-21T12:49:02.000-04:00', 'claudia', 123453191, 'Virginia Street,701')]


In [81]:
# 3. Generate a monthly bill for a credit card number for a given month and year.

def getMonthlyBillForCreditCardBYMonthAndYear(month, year):
    
    sqlStatement5 = "SELECT CREDIT_CARD_NO, ROUND(SUM(TRANSACTION_VALUE),2) AS \
        TOTAL_TRANSACTION \
            FROM cdw_sapp_credit_card \
            WHERE MONTH(STR_TO_DATE(TIMEID, '%Y%m%d')) = '{}' \
                AND YEAR(STR_TO_DATE(TIMEID, '%Y%m%d')) = '{}' \
            GROUP BY CREDIT_CARD_NO"
    
    cursor.execute(sqlStatement5.format(month, year))

    result5 = cursor.fetchall()
    print(result5)

In [ ]:
month = input("Enter the month for which you would like the transaction records in a numberic format (1-12):")
year = input("Enter the year for which you would like the transaction records: ")
getMonthlyBillForCreditCardBYMonthAndYear(month, year)

In [83]:
# 4. Display the transactions made by a customer between two dates. 
# Order by year, month, and day in descending order.

def getTransactionsByCustomerForTwoDates(date1, date2, SSN):

	sqlStatement6 = "SELECT DISTINCT tran.TRANSACTION_ID, tran.TRANSACTION_VALUE, \
		cust.FIRST_NAME, cust.LAST_NAME FROM cdw_sapp_credit_card tran \
			INNER JOIN cdw_sapp_customer cust \
				ON cust.SSN = tran.CUST_SSN \
			WHERE (tran.TIMEID BETWEEN '{}' AND '{}') AND tran.Cust_SSN = {} \
				ORDER BY YEAR(STR_TO_DATE(tran.TIMEID, '%Y%m%d')) DESC, \
						MONTH(STR_TO_DATE(tran.TIMEID, '%Y%m%d')) DESC, \
						DAY(STR_TO_DATE(tran.TIMEID, '%Y%m%d')) DESC"
	
	cursor.execute(sqlStatement6.format(date1, date2, SSN))
	
	result5 = cursor.fetchall()
	print(result5)

In [84]:
date1 = input("Please enter first date (example:20180824): ")
date2 = input("Please enter second date (example:20180824): ")
SSN = input("Please enter Customer SSN: ")
getTransactionsByCustomerForTwoDates(date1, date2, SSN)

[(22598, 59.59, 'Harrison', 'King'), (22587, 86.78, 'Harrison', 'King'), (22612, 89.17, 'Harrison', 'King')]
